In [24]:
import requests
import json
import os

In [36]:
API_URL = "API_URL"

In [38]:
def add_conversation_to_memory(user_input: str, assistant_response: str, user_id: str = "default"):
  """記憶サービスに会話履歴を追加する"""
  url = f"{API_URL}/api/add"
  payload = {
    "user_input": user_input,
    "assistant_response": assistant_response,
    "user_id": user_id
  }
  try:
    response = requests.post(url, json=payload)
    response.raise_for_status()
    return response.json()
  except requests.exceptions.RequestException as e:
    print(f"会話の追加に失敗しました: {e}")
    return None

def query_memory_for_similar_conversations(query: str, user_id: str = "default", top_k: int = 3):
  """記憶サービスから類似の会話履歴を検索する"""
  url = f"{API_URL}/api/query"
  payload = {
    "query": query,
    "user_id": user_id,
    "top_k": top_k
  }
  try:
    response = requests.post(url, json=payload)
    response.raise_for_status()
    return response.json()
  except requests.exceptions.RequestException as e:
    print(f"記憶の検索に失敗しました: {e}")
    return None

def get_memory_stats():
  """記憶サービスの統計情報を取得する"""
  url = f"{API_URL}/api/stats"
  try:
    response = requests.get(url)
    response.raise_for_status()
    return response.json()
  except requests.exceptions.RequestException as e:
    print(f"統計情報の取得に失敗しました: {e}")
    return None

def health_check():
  health_url = f"{API_URL}/api/health"
  try:
    response = requests.get(health_url)
    response.raise_for_status()
    health_status = response.json()
    print(f"ステータス: {json.dumps(health_status, indent=2)}")
    return health_status.get("status") == "healthy"
  except requests.exceptions.RequestException as e:
    print(f"失敗: {e}")
    return False

In [40]:
# いくつかの会話を記憶に追加する
add_result1 = add_conversation_to_memory("私はりんごを食べるのが好きです。", "はい、わかりました。", user_id="user123")
print(f"追加結果1: {json.dumps(add_result1, indent=2)}")

add_result2 = add_conversation_to_memory("ルビィちゃん！", "はい～", user_id="user123")
print(f"追加結果2: {json.dumps(add_result2, indent=2)}")

add_result3 = add_conversation_to_memory("何が好き?", "チョコミントよりも あ・な・た！", user_id="user456")
print(f"追加結果3: {json.dumps(add_result3, indent=2)}")

追加結果1: {
  "message": "\u4f1a\u8a71\u5c65\u6b74\u304c\u8ffd\u52a0\u3055\u308c\u307e\u3057\u305f",
  "status": "success",
  "timestamp": "2025-06-04T19:37:16.821870",
  "user_id": "user123"
}
追加結果2: {
  "message": "\u4f1a\u8a71\u5c65\u6b74\u304c\u8ffd\u52a0\u3055\u308c\u307e\u3057\u305f",
  "status": "success",
  "timestamp": "2025-06-04T19:37:17.003850",
  "user_id": "user123"
}
追加結果3: {
  "message": "\u4f1a\u8a71\u5c65\u6b74\u304c\u8ffd\u52a0\u3055\u308c\u307e\u3057\u305f",
  "status": "success",
  "timestamp": "2025-06-04T19:37:17.189225",
  "user_id": "user456"
}


In [42]:
# 類似する会話を検索
print("user123：")
query_result1 = query_memory_for_similar_conversations("私はどんな果物が一番好き？", user_id="user123", top_k=1)  # 最も類似する結果を返す
print(f"{json.dumps(query_result1, indent=2)}")

print("\nuser456：")
query_result2 = query_memory_for_similar_conversations("あなたの名前は何ですか", user_id="user456", top_k=1)
print(f"{json.dumps(query_result2, indent=2)}")

print("\n関連する記憶がない会話を検索")
query_result3 = query_memory_for_similar_conversations("ピカチュウ", user_id="user123", top_k=1)
print(f"{json.dumps(query_result3, indent=2)}")

user123：
{
  "found_memories": 1,
  "has_relevant_memory": true,
  "memories": [
    {
      "assistant_response": "\u306f\u3044\u3001\u308f\u304b\u308a\u307e\u3057\u305f\u3002",
      "id": 20,
      "similarity": 0.8011915683746338,
      "timestamp": "2025-06-04T19:36:44.662030",
      "user_id": "user123",
      "user_input": "\u79c1\u306f\u308a\u3093\u3054\u3092\u98df\u3079\u308b\u306e\u304c\u597d\u304d\u3067\u3059\u3002"
    }
  ],
  "query": "\u79c1\u306f\u3069\u3093\u306a\u679c\u7269\u304c\u4e00\u756a\u597d\u304d\uff1f",
  "timestamp": "2025-06-04T19:40:27.595142",
  "user_id": "user123"
}

user456：
{
  "found_memories": 1,
  "has_relevant_memory": true,
  "memories": [
    {
      "assistant_response": "\u30c1\u30e7\u30b3\u30df\u30f3\u30c8\u3088\u308a\u3082 \u3042\u30fb\u306a\u30fb\u305f\uff01",
      "id": 22,
      "similarity": 0.7098116278648376,
      "timestamp": "2025-06-04T19:36:45.034197",
      "user_id": "user456",
      "user_input": "\u4f55\u304c\u597d\u304d?"
   

In [43]:
# 統計情報を取得
stats = get_memory_stats()
print(json.dumps(stats, indent=2))

{
  "max_memory_items": 100,
  "similarity_threshold": 0.7,
  "total_conversations": 26,
  "total_embeddings": 26
}


In [44]:
health_check()

ステータス: {
  "service": "Memory API",
  "status": "healthy",
  "timestamp": "2025-06-04T19:40:51.622284"
}


True

In [30]:
import google.generativeai as genai
import time

In [11]:
GEMINI_API_KEY = "your_gemini_api"

In [33]:
genai.configure(api_key=GEMINI_API_KEY)
GEMINI_MODEL = genai.GenerativeModel('gemini-1.5-flash')

In [45]:
def chat_with_gemini_and_memory(user_input: str, user_id: str = "default"):
  """
  記憶 + Gemini API
  Args:
    user_input (str): ユーザーの入力
    user_id (str): ユーザー ID
  Returns:
    str: Gemini モデルからの返答
  """
  print(f"\n[{user_id}] 用戶輸入: {user_input}")

  # 記憶サービスを検索
  query_results = query_memory_for_similar_conversations(user_input, user_id, top_k=2) # 関連する記憶を 2 件取得する

  context_from_memory = ""
  if query_results and query_results.get('has_relevant_memory', False):
    memories = query_results['memories']
    print(f"{len(memories)} 件の関連する記憶が見つかりました")
    context_from_memory = "以下は、過去に user と交わされた会話の一部で、返答に役立つかもしれません：\n"
    for i, mem in enumerate(memories):
      context_from_memory += f" 会話記録: {i+1}:\n"
      context_from_memory += f"user: {mem['user_input']}\n"
      context_from_memory += f"あなた: {mem['assistant_response']}\n"
  else:
    print("関連する記憶が見つかりませんでした")

  # 記憶の文脈と現在の入力を統合する
  full_prompt = f"""
{context_from_memory}
---次の入力は以下です:---\n
{user_input}
"""
  print(f"--- Gemini に送信するプロンプト ---")
  print(full_prompt)
  print(f"----------------------------")

  # Gemini API を呼び出す
  try:
    response = GEMINI_MODEL.generate_content(full_prompt)
    gemini_response = response.text
    print(f"Gemini 回應:\n {gemini_response}\n")

    # 現在の会話を記憶サービスに保存
    add_conversation_to_memory(user_input, gemini_response, user_id)

    return gemini_response

  except Exception as e:
    print(f"Gemini API エラーまたは内容がブロックされました: {e}")
    return "返答が得られませんでした"

In [46]:
if __name__ == "__main__":
  print("「exit」または「quit」と入力すると終了します。\n")
  current_user_id = "user123"
  print(f"現在のユーザーID: {current_user_id}")

  while True:
    user_input_text = input("あなたの入力: ")
    if user_input_text.lower() in ["exit", "quit"]:
      print("対話を終了します。")
      break

    gemini_response = chat_with_gemini_and_memory(user_input_text, current_user_id)

    time.sleep(1)

「exit」または「quit」と入力すると終了します。

現在のユーザーID: user123
あなたの入力: 私が好きな食べ物は何ですか？

[user123] 用戶輸入: 私が好きな食べ物は何ですか？
2 件の関連する記憶が見つかりました
--- Gemini に送信するプロンプト ---

以下は、過去に user と交わされた会話の一部で、返答に役立つかもしれません：
 会話記録: 1:
user: 私はりんごを食べるのが好きです。
あなた: はい、わかりました。
 会話記録: 2:
user: 私はりんごを食べるのが好きです。
あなた: はい、わかりました。

---接下來是新的輸入:---

私が好きな食べ物は何ですか？

----------------------------
Gemini 回應:
 会話ログからは、あなたがリンゴが好きだという情報しか得られません。そのため、あなたが好きな食べ物は**リンゴ**です。


あなたの入力: exit
対話を終了します。
